# TODO: 
# - delete all unused datasets

In [29]:
import numpy as np
import pandas as pd

In [30]:
#berlin_listings = pd.read_csv("./data/berlin/listings.csv.gz")
#berlin_calendar = pd.read_csv("./data/berlin/calendar.csv.gz")
#berlin_reviews = pd.read_csv("./data/berlin/reviews.csv.gz")

### Read AirBnB data of Munich, Germany
##### Time period: 15.03.2019 + 1 year in the future (calendar)

Problem of the dataset is the timeperiod

In [31]:
munich_listings_raw = pd.read_csv("./data/munich/listings.csv.gz") 

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (43,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
#munich_calendar = pd.read_csv("./data/munich/calendar.csv.gz")

In [33]:
#munich_reviews = pd.read_csv("./data/munich/reviews.csv.gz")

In [34]:
#munich_neighborhoods = pd.read_csv("./data/munich/neighbourhoods.csv")

## 1. Price prediction 
Predict the price with an Linear Regression Model with the data from the listing. Discover which of the following factors influce the price the most
- location = neighbourhood + latitude + longitude
- room-type = room_type
- rating
- host - super-host and identified
- instant bookable

#### Dataset preparation

In [35]:
# define all necessary categories
categories = ["id",
              "price",
              "neighbourhood",
              "latitude",
              "longitude", 
              "review_scores_value",
              "review_scores_rating", 
              "number_of_reviews",
              "room_type", 
              "host_identity_verified",
              "host_is_superhost", 
              "instant_bookable" ]
# create dataset only with necessary variables/categories
munich_listings = munich_listings_raw[categories]
munich_listings.head()

,id,price,neighbourhood,latitude,longitude,review_scores_value,review_scores_rating,number_of_reviews,room_type,host_identity_verified,host_is_superhost,instant_bookable
0,36720,$95.00,Ludwigsvorstadt - Isarvorstadt,48.13057,11.56929,9.0,98.0,25,Entire home/apt,f,f,f
1,97945,$80.00,Hadern,48.11476,11.48782,9.0,97.0,115,Entire home/apt,f,t,f
2,114695,$95.00,Berg am Laim,48.11923,11.63726,9.0,95.0,47,Entire home/apt,t,f,f
3,127383,$120.00,Maxvorstadt,48.15198,11.56486,10.0,97.0,75,Entire home/apt,t,t,f
4,159634,$55.00,Pasing - Obermenzing,48.13898,11.46612,9.0,93.0,27,Entire home/apt,t,f,t


#### Check for missing values

In [44]:
def find_categories_w_missing_values(df):
    categories_w_mising_values = [] 
    for category in munich_listings.columns:
        if not munich_listings[category].isnull().values.any():
            print("No missing values in" + category)
        else:
            print("\033[1m" + "Missing values in", category + "\033[0m")
            categories_w_mising_values.append(category)
    print("\033[1m" + "\nThere are", len(categories_w_mising_values), "categories with missing values" + "\033[0m")
    return categories_w_mising_values

In [37]:
categories_w_mising_values = find_categories_w_missing_values(munich_listings)

No missing values inid
No missing values inprice
No missing values inneighbourhood
No missing values inlatitude
No missing values inlongitude
Missing values in review_scores_value
Missing values in review_scores_rating
No missing values innumber_of_reviews
No missing values inroom_type
Missing values in host_identity_verified
Missing values in host_is_superhost
No missing values ininstant_bookable

There are 4  categories with missing values


#### Dealing with the missing values in columns review_scores_values, host_identity_verified and host_is_superhost

In [38]:
# count missing values
def print_missing_values(df, categories):
    """ Args:
        df: 
        categories:
        
        return:
        none
    """
    for category in categories:
        print(category)
        print(df[category].isnull().value_counts(),"\n")

In [39]:
print_missing_values(munich_listings, categories_w_mising_values)

review_scores_value
False    7330
True     2528
Name: review_scores_value, dtype: int64 

review_scores_rating
False    7341
True     2517
Name: review_scores_rating, dtype: int64 

host_identity_verified
False    9852
True        6
Name: host_identity_verified, dtype: int64 

host_is_superhost
False    9852
True        6
Name: host_is_superhost, dtype: int64 



About 34% of the dataset entries have missing values in the rating categories. The assumption is that these are the listings without any review. Thus the review will be used for the price prediction all these rows will be removed from the dataset, because imputing the reviews would not make much sense.  

In [40]:
# drop all listings with zero reviews
munich_listings = munich_listings.drop(munich_listings[munich_listings["number_of_reviews"]==0].index)
print_missing_values(munich_listings, categories_w_mising_values)

review_scores_value
False    7330
True      185
Name: review_scores_value, dtype: int64 

review_scores_rating
False    7341
True      174
Name: review_scores_rating, dtype: int64 

host_identity_verified
False    7513
True        2
Name: host_identity_verified, dtype: int64 

host_is_superhost
False    7513
True        2
Name: host_is_superhost, dtype: int64 



There are only a few rows (~2%) left with missing value - these rows will also be dropped.

In [45]:
for category in categories_w_mising_values:
    munich_listings = munich_listings.dropna(subset=[category], axis=0)
find_categories_w_missing_values(munich_listings)

No missing values inid
No missing values inprice
No missing values inneighbourhood
No missing values inlatitude
No missing values inlongitude
No missing values inreview_scores_value
No missing values inreview_scores_rating
No missing values innumber_of_reviews
No missing values inroom_type
No missing values inhost_identity_verified
No missing values inhost_is_superhost
No missing values ininstant_bookable

There are 0 categories with missing values


[]

#### Exploring categorial variables

In [46]:
room_types = munich_listings["room_type"].unique()
print("There are", len(room_types), "unique room types:")
for room_type in room_types:
    print("*", room_type)

There are 3 unique room types:
* Entire home/apt
* Private room
* Shared room


In [47]:
neighbourhoods = munich_listings["neighbourhood"].unique()
print("There are", len(neighbourhoods), "unique neighbourhoods:")
for neighbourhood in neighbourhoods:
    print("*", neighbourhood)

There are 45 unique neighbourhoods:
* Ludwigsvorstadt - Isarvorstadt
* Hadern
* Berg am Laim
* Maxvorstadt
* Pasing - Obermenzing
* Sendling - Westpark
* Untergiesing - Harlaching
* Schwabing - West
* Englschalking
* Au - Haidhausen
* Neuhausen - Nymphenburg
* Schwanthalerhöhe
* Bogenhausen
* Trudering - Riem
* Sendling
* Ramersdorf - Perlach
* Schwabing - Freimann
* Feldmoching - Hasenbergl
* Altbogenhausen
* Thalkirchen - Obersendling - Forstenried - Fürstenried - Solin
* Münchner Freiheit
* Am Riesenfeld
* Neuhausen
* Obergiesing
* Altstadt - Lehel
* Laim
* Aubing - Lochhausen - Landwied
* Milbertshofen - Am Hart
* Daglfing
* Schwabing - Ost
* Parkstadt
* München - Moosach
* Alte Heide - Hirschau
* Neufreimann
* Allach - Untermenzing
* Alt Moosach
* Freimann
* Nymphenburg
* Am Hart
* Oberföhring
* Milbertshofen
* Obere Isarau
* Herzogpark
* Johanneskirchen
* Kleinhesselohe


### 2. Reviews
 Is there any connection between price and rating (are the more expensiver offers better reviewd?)
 which was mentioned at most in good ratings?!

### 3. Influence of events during the year, e.g. Oktoberfest
 - problem: only prices from the future, the real bookings are not available
 - check data from march against data from spetember and october and check number of available offers, price, availability
 
 